Author: Thais Rodrigues Neubauer

This code evaluate the statistic relevance of the differences in regressors' predictions. Adapt the value of the variables in section "Define global variables" to properly define paths and to control the characteristics of the evaluation pipeline.

The code was created to be executed using Google Colab. If that's not the case, delete the last two lines of the "Import required libraries" section.

#Import libraries

In [ ]:
import pandas as pd
import scipy.stats as stats
import numpy as np
import seaborn as sns
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
import os

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
###########################
# When using Google Colab #
###########################
from google.colab import files, drive
drive.mount('/content/gdrive/')

#Global variables


In [ ]:
EVENT_LOG_ID = 'helpdesk' #incidents  #define models corresponding to which event log will be evaluated
CASE_COL = 'CaseID' #'number'         #inform the column that corresponds to the case id in the event log
MODELS_DIR = 'LSTM100/'
DIR_PATH = f'/content/gdrive/My Drive/ProcessMiningResearch/Interactive_trace_clustering/experiments_log_incidentes/rt_prediction/LSTM/{EVENT_LOG_ID}/output_files/results/'

P_TRASHOLD = 0.05

# Load data

In [ ]:
results = [f for f in os.listdir(f'{DIR_PATH}{MODELS_DIR}') if '.csv' in f and 'TRF_1-EIF_1' in f]
df = pd.read_csv(f'{DIR_PATH}{MODELS_DIR}{results[0]}')[[CASE_COL, 'Prefix length', 'Ground truth']]

for result in results:
  df[result.replace('.csv','')] = pd.read_csv(f'{DIR_PATH}{MODELS_DIR}{result}')['Predicted times']

df = df.rename(columns={CASE_COL:'case_id','Prefix length': 'prefixes_len', 'Ground truth':'y_test'})
df.head()

,case_id,prefixes_len,y_test,incidents-LSTM-TRF_1-EIF_1#RUN0,incidents-LSTM-TRF_1-EIF_1#RUN1,incidents-LSTM-TRF_1-EIF_1#RUN2,incidents-LSTM-TRF_1-EIF_1-5d_P3.0-time_annotated--evntTrue_evntposFalse_timesFalse_attribsFalse#RUN2#RUN0,incidents-LSTM-TRF_1-EIF_1-5d_P3.0-time_annotated--evntTrue_evntposFalse_timesFalse_attribsFalse#RUN2#RUN1,incidents-LSTM-TRF_1-EIF_1-5d_P3.0-time_annotated--evntTrue_evntposFalse_timesFalse_attribsFalse#RUN2#RUN2,incidents-LSTM-TRF_1-EIF_1-8d_P3.0-time_annotated--evntTrue_evntposFalse_timesFalse_attribsFalse#RUN2#RUN0,...,incidents-LSTM-TRF_1-EIF_1-15d_P3.0-time_annotated--evntTrue_evntposFalse_timesFalse_attribsFalse#RUN1#RUN2,incidents-LSTM-TRF_1-EIF_1-15d_P3.0-time_annotated--evntTrue_evntposFalse_timesFalse_attribsFalse#RUN2#RUN0,incidents-LSTM-TRF_1-EIF_1-15d_P3.0-time_annotated--evntTrue_evntposFalse_timesFalse_attribsFalse#RUN2#RUN1,incidents-LSTM-TRF_1-EIF_1-15d_P3.0-time_annotated--evntTrue_evntposFalse_timesFalse_attribsFalse#RUN2#RUN2,incidents-LSTM-TRF_1-EIF_1-12d_P3.0-time_annotated--evntTrue_evntposFalse_timesFalse_attribsFalse#RUN2#RUN0,incidents-LSTM-TRF_1-EIF_1-12d_P3.0-time_annotated--evntTrue_evntposFalse_timesFalse_attribsFalse#RUN2#RUN1,incidents-LSTM-TRF_1-EIF_1-12d_P3.0-time_annotated--evntTrue_evntposFalse_timesFalse_attribsFalse#RUN2#RUN2,incidents-LSTM-TRF_1-EIF_1-12d_P3.0-time_annotated--evntTrue_evntposFalse_timesFalse_attribsFalse#RUN1#RUN0,incidents-LSTM-TRF_1-EIF_1-12d_P3.0-time_annotated--evntTrue_evntposFalse_timesFalse_attribsFalse#RUN1#RUN1,incidents-LSTM-TRF_1-EIF_1-12d_P3.0-time_annotated--evntTrue_evntposFalse_timesFalse_attribsFalse#RUN1#RUN2
0,INC0023795,2,433080.0,456728.7,464964.28,468887.97,453842.53,453842.53,453842.53,456604.80,...,447173.84,452844.90,450153.20,453742.10,444222.90,444141.97,445566.00,442190.20,448362.44,447381.66
1,INC0023796,2,781920.0,862594.3,896238.75,869660.94,901195.70,901195.70,901195.70,904002.10,...,878800.50,884836.90,870777.25,874362.70,881573.20,871113.90,875582.30,882470.06,880075.70,883861.30
2,INC0023796,3,778260.0,856235.1,880701.00,851469.80,897419.00,897419.00,897419.00,881808.75,...,886327.80,888183.44,885005.75,865619.90,876298.60,876989.50,885029.75,877394.75,876203.70,897700.90
3,INC0023796,4,713220.0,854801.4,877239.70,871284.50,919466.44,919466.44,919466.44,884062.44,...,894671.06,894638.50,886665.56,876701.20,893629.06,893483.10,907431.70,882857.94,898008.40,903002.80
4,INC0023796,5,713220.0,959648.6,960005.50,964608.25,968765.20,968765.20,968765.20,961573.90,...,973306.20,962442.44,911335.75,934732.25,979374.60,949529.50,961553.00,944070.90,969542.75,952544.44


#Check differences among predictions

Test per individual prediction

In [ ]:
cols = list(df.columns)
cols.remove('case_id')
cols.remove('prefixes_len')
cols.remove('y_test')
len(cols)

57

In [ ]:
#checking if any model's answers does follow a normal distribution
#(if it does, we should not apply wilcoxon)
normally_distrib = []
for col in cols:
  if(stats.shapiro(df[col])[1]) > P_TRASHOLD:
    normally_distrib.append(col)
print('Models with normally distributed averages:' + str(normally_distrib))

Models with normally distributed averages:[]


In [ ]:
test_values = {} #keep record of the ones that NO STATISTICAL DIFFERENCE WAS FOUND!
for i,col1 in enumerate(cols):
  for col2 in cols[i+1:]:
    if col1 in normally_distrib and col2 in normally_distrib:
      pvalue=stats.ttest_ind(df[col1], df[col2])[1]
      if pvalue > P_TRASHOLD:
        test_values[f'{col1}#{col2}'] = pvalue
    else:
      if df[col1].equals(df[col2]):
        continue
      pvalue=stats.wilcoxon(df[col1], df[col2])[1]
      if pvalue > P_TRASHOLD:
        test_values[f'{col1}#{col2}'] = pvalue

In [ ]:
test_values

{'incidents-LSTM-TRF_1-EIF_1#RUN1#incidents-LSTM-TRF_1-EIF_1-8d_P3.0-time_annotated--evntTrue_evntposFalse_timesFalse_attribsFalse#RUN2#RUN2': 0.23005665491149285,
 'incidents-LSTM-TRF_1-EIF_1#RUN1#incidents-LSTM-TRF_1-EIF_1-8d_P3.0-time_annotated--evntTrue_evntposFalse_timesFalse_attribsFalse#RUN1#RUN0': 0.587787411799777,
 'incidents-LSTM-TRF_1-EIF_1#RUN1#incidents-LSTM-TRF_1-EIF_1-10d_P3.0-time_annotated--evntTrue_evntposFalse_timesFalse_attribsFalse#RUN1#RUN0': 0.09659977379378626,
 'incidents-LSTM-TRF_1-EIF_1#RUN1#incidents-LSTM-TRF_1-EIF_1-12d_P3.0-time_annotated--evntTrue_evntposFalse_timesFalse_attribsFalse#RUN0': 0.44836225740892366,
 'incidents-LSTM-TRF_1-EIF_1#RUN1#incidents-LSTM-TRF_1-EIF_1-15d_P3.0-time_annotated--evntTrue_evntposFalse_timesFalse_attribsFalse#RUN2': 0.29724226344415106,
 'incidents-LSTM-TRF_1-EIF_1#RUN2#incidents-LSTM-TRF_1-EIF_1-8d_P3.0-time_annotated--evntTrue_evntposFalse_timesFalse_attribsFalse#RUN2#RUN2': 0.18088019081539464,
 'incidents-LSTM-TRF_1-EI